In [1]:
import torch
from torch import nn
import numpy as np
from torchsummary import summary
import Lossv2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
summary(Lossv2.Loss().to(device), (6, 768, 512))  ###  IDK why, but I have to switch the dims in AvgPool on Loss() to get this to work :/ But that breaks the stuff bellow

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 768, 512]             224
            Conv2d-2         [-1, 32, 384, 512]             128
            Conv2d-3         [-1, 32, 384, 256]             128
               ELU-4         [-1, 32, 384, 256]               0
            Conv2d-5         [-1, 32, 384, 256]           1,056
            Conv2d-6         [-1, 32, 384, 256]             128
            Conv2d-7         [-1, 32, 384, 256]             128
               ELU-8         [-1, 32, 384, 256]               0
            Conv2d-9         [-1, 32, 384, 256]           1,056
           Conv2d-10         [-1, 32, 192, 256]             128
           Conv2d-11         [-1, 32, 192, 128]             128
              ELU-12         [-1, 32, 192, 128]               0
           Conv2d-13         [-1, 32, 384, 256]           1,056
        MaxPool2d-14         [-1, 32, 1

In [1]:
import shutil
import torch
import torch.nn as nn
import numpy as np
torch.manual_seed(100)
np.random.seed(100)
import Lossv2
import generateLossImages
import time

global counter 
counter = 0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def save_ckp(state, is_best, checkpoint_dir="./models/rest/", best_model_dir="./models/best/"):
    global counter 
    f_path = checkpoint_dir + str(counter) + '_checkpoint.pt'
    counter = counter + 1
    torch.save(state, f_path)
    if is_best:
        best_fpath = best_model_dir + 'best_model.pt'
        shutil.copyfile(f_path, best_fpath)

def load_ckp(model, optimizer, checkpoint_fpath="./models/best/best_model.pt"):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model, optimizer, checkpoint['epoch'], checkpoint['index'], checkpoint['min_lr'], checkpoint['max_lr'], checkpoint['steps'], checkpoint['step_size'], checkpoint['falling'], checkpoint['threshold']

In [2]:

loss_fn = nn.L1Loss()
min_lr = 0.0003
max_lr = 0.001
decay = 0.8
steps = 300
falling = True
start_epoch = 0
start_index = 0
threshold = [0.24, 0.2, 0.16, 0.12, 0.09, 0.06, 0.04, 0.03, 0.02, 0.015, 0.01 -1]
step_size = (max_lr-min_lr)/steps
model = Lossv2.Loss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=max_lr, momentum=0.96)

In [2]:
loss_fn = nn.L1Loss()
model = Lossv2.Loss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.96)
model, optimizer, start_epoch, start_index, min_lr, max_lr, steps, step_size, falling, threshold = load_ckp(model, optimizer)

In [1]:
li = [31.007527351379395, 0.01501011848449707, 0.0010006427764892578, 28.98952603340149, 0.48702239990234375, 47.146605014801025, 12.32689380645752, 0.5665144920349121]
su = sum(li)
print([100*l/su for l in li])


[25.72382749606882, 0.012452385971162576, 0.0008301326924880367, 24.049694721718666, 0.40403351288996003, 39.11279737602433, 10.22638426619226, 0.4699801084423085]


In [3]:
printing = True


running_loss = 0.
last_loss = 0.
lis = []
 # might have to delete when loading model
threshold_decay = 0.2
flags = [True for _ in threshold]
record = 1
epochs = 100

times = [0]*10
transforms = 117
for epoch in range(start_epoch, epochs):

    training = generateLossImages.MakeIter(generateLossImages.get_image_pairs_transforms_with_loss(cupy=True, start=start_index if epoch == start_epoch else 0)) # start=2*71*50*4))#start_index if epoch == start_epoch else 0)) 
    val = generateLossImages.get_image_pairs_transforms_with_loss("C:/Users/Rani/Desktop/ai_val/16")
    training_loader = torch.utils.data.DataLoader(training)
    for index, data in enumerate(training_loader): # loading data takes 25.72% of the time

        inputs, labels = data  #0.01%

        labels = labels.to("cuda") # 0.00%

        # Make predictions for this batch
        outputs = model(inputs) # 24.05%

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels) # 0.40%

        loss.backward() # 39.11%

        # Gather data and report
        running_loss += loss.item() # 10.23%

        if printing: # From here down only takes 0.47% of the time
            lis.append((f'{outputs.item():.3}',f'{labels.item():.3}'))
        if index % transforms == transforms-1:
            # Zero your gradients for every batch!
            # Adjust learning weights
            last_loss = running_loss / transforms # loss per batch
            if last_loss < threshold[0] and flags[0]:
                threshold = threshold[1:]
                flags = flags[1:]
                flag1 = False
                min_lr *= threshold_decay
                max_lr *= threshold_decay
                step_size = (max_lr-min_lr)/steps
                falling = True
                optimizer.param_groups[-1]['lr'] = max_lr
                if printing:
                    print(min_lr)
                    print(max_lr)
                    print(steps)
                    print(step_size)
                checkpoint = {
                    'epoch': epoch,
                    'index': index,
                    'min_lr': min_lr,
                    'max_lr': max_lr,
                    'steps': steps,
                    'step_size': step_size,
                    'falling': falling,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'threshold': threshold
                }
                save_ckp(checkpoint, True)
            elif falling:
                optimizer.param_groups[-1]['lr'] = optimizer.param_groups[-1]['lr'] - step_size
                if optimizer.param_groups[-1]['lr'] < min_lr:
                    falling = False
                    max_lr *= decay
                    min_lr *= decay
                    steps /= decay
                    step_size = (max_lr-min_lr)/steps

                    if printing:
                        print(min_lr)
                        print(max_lr)
                        print(steps)
                        print(step_size)
                    
                    
                    checkpoint = {
                        'epoch': epoch,
                        'index': index,
                        'min_lr': min_lr,
                        'max_lr': max_lr,
                        'steps': steps,
                        'step_size': step_size,
                        'falling': falling,
                        'state_dict': model.state_dict(),
                        'optimizer': optimizer.state_dict(),
                        'threshold': threshold
                    }
                    save_ckp(checkpoint, False)

            else: 
                optimizer.param_groups[-1]['lr'] += step_size
                if optimizer.param_groups[-1]['lr'] > max_lr:
                    falling = True


            
            optimizer.step()
            optimizer.zero_grad()
            if printing:
                print(optimizer.param_groups[-1]['lr'])
                print('  batch {} loss: {}'.format(index + 1, last_loss))
                print(lis)
                lis = []
            running_loss = 0

            if last_loss < record:
                record = last_loss
                if last_loss < 0.08:
                    checkpoint = {
                    'epoch': epoch,
                    'index': index,
                    'min_lr': min_lr,
                    'max_lr': max_lr,
                    'steps': steps,
                    'step_size': step_size,
                    'falling': falling,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'threshold': threshold
                }
                    save_ckp(checkpoint, True)

c:\Users\Rani\anaconda3\envs\torch\lib\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


0.0009976666666666667
  batch 117 loss: 0.2686974798512255
[('0.506', '0.001'), ('0.506', '0.001'), ('0.506', '0.001'), ('0.506', '0.001'), ('0.506', '0.001'), ('0.506', '0.001'), ('0.506', '0.001'), ('0.507', '0.001'), ('0.506', '0.001'), ('0.511', '0.001'), ('0.508', '0.999'), ('0.51', '0.999'), ('0.515', '0.999'), ('0.512', '0.999'), ('0.511', '0.999'), ('0.519', '0.999'), ('0.518', '0.999'), ('0.511', '0.999'), ('0.502', '0.999'), ('0.505', '0.999'), ('0.511', '0.999'), ('0.498', '0.999'), ('0.506', '0.2'), ('0.506', '0.2'), ('0.506', '0.2'), ('0.506', '0.2'), ('0.506', '0.2'), ('0.506', '0.2'), ('0.508', '0.2'), ('0.506', '0.2'), ('0.511', '0.2'), ('0.508', '0.2'), ('0.508', '0.2'), ('0.513', '0.6'), ('0.511', '0.6'), ('0.512', '0.6'), ('0.509', '0.6'), ('0.509', '0.6'), ('0.515', '0.6'), ('0.517', '0.6'), ('0.508', '0.6'), ('0.514', '0.6'), ('0.515', '0.6'), ('0.513', '0.6'), ('0.513', '0.6'), ('0.508', '0.4'), ('0.507', '0.4'), ('0.507', '0.4'), ('0.507', '0.4'), ('0.507', '0.4'

In [8]:
printing = False


running_loss = 0.
last_loss = 0.
lis = []
 # might have to delete when loading model
threshold_decay = 0.2
flags = [True for _ in threshold]
record = 1
epochs = 100

times = [0]*8
for epoch in range(start_epoch, epochs):

    training = generateLossImages.MakeIter(generateLossImages.get_image_pairs_transforms_with_loss(cupy=True, start=start_index if epoch == start_epoch else 0)) # start=2*71*50*4))#start_index if epoch == start_epoch else 0)) 
    val = generateLossImages.get_image_pairs_transforms_with_loss("C:/Users/Rani/Desktop/ai_val/16")
    training_loader = torch.utils.data.DataLoader(training)
    end = 0
    for index, data in enumerate(training_loader):#[90.06174230575562, 0.03202486038208008, 0.016014575958251953, 84.36967897415161, 1.3647308349609375, 137.4005196094513, 34.81299662590027, 1.9427943229675293, 0, 0]
        start = time.time()
        if end != 0:
            times[0] += (start-end)
        inputs, labels = data  

        load = time.time()
        times[1] += load-start

        labels = labels.to("cuda")

        lab = time.time()
        times[2] += (lab-load)

        # Make predictions for this batch
        outputs = model(inputs)

        out = time.time()
        times[3] += (out-lab)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)

        los = time.time()
        times[4] += (los-out)

        loss.backward()

        back = time.time()
        times[5] += (back-los)
        # Gather data and report
        running_loss += loss.item()
        
        runn = time.time()
        times[6] += (runn-back)

        if printing:
            lis.append((f'{outputs.item():.3}',f'{labels.item():.3}'))
        if index % 71 == 70:
            # Zero your gradients for every batch!
            # Adjust learning weights
            last_loss = running_loss / 71 # loss per batch
            if last_loss < threshold[0] and flags[0]:
                threshold = threshold[1:]
                flags = flags[1:]
                flag1 = False
                min_lr *= threshold_decay
                max_lr *= threshold_decay
                step_size = (max_lr-min_lr)/steps
                falling = True
                optimizer.param_groups[-1]['lr'] = max_lr
                if printing:
                    print(min_lr)
                    print(max_lr)
                    print(steps)
                    print(step_size)
                checkpoint = {
                    'epoch': epoch,
                    'index': index,
                    'min_lr': min_lr,
                    'max_lr': max_lr,
                    'steps': steps,
                    'step_size': step_size,
                    'falling': falling,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'threshold': threshold
                }
                save_ckp(checkpoint, True)
            elif falling:
                optimizer.param_groups[-1]['lr'] = optimizer.param_groups[-1]['lr'] - step_size
                if optimizer.param_groups[-1]['lr'] < min_lr:
                    falling = False
                    max_lr *= decay
                    min_lr *= decay
                    steps /= decay
                    step_size = (max_lr-min_lr)/steps

                    if printing:
                        print(min_lr)
                        print(max_lr)
                        print(steps)
                        print(step_size)

            else: 
                optimizer.param_groups[-1]['lr'] += step_size
                if optimizer.param_groups[-1]['lr'] > max_lr:
                    falling = True


            
            optimizer.step()
            optimizer.zero_grad()
            if printing:
                print(optimizer.param_groups[-1]['lr'])
                print('  batch {} loss: {}'.format(index + 1, last_loss))
                print(lis)
                lis = []
            running_loss = 0

            if last_loss < record:
                record = last_loss
                if last_loss < 0.08:
                    checkpoint = {
                    'epoch': epoch,
                    'index': index,
                    'min_lr': min_lr,
                    'max_lr': max_lr,
                    'steps': steps,
                    'step_size': step_size,
                    'falling': falling,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'threshold': threshold
                }
                    save_ckp(checkpoint, True)
            print(times)
        end = time.time()
        times[7] += (end-runn)
                

C:/Users/Rani/Desktop/ai_training_immages/1/_DSC0001.NEF
[1.5390501022338867, 0.0010004043579101562, 0.0, 1.6169846057891846, 0.030069828033447266, 2.6113336086273193, 0.696631669998169, 0.0]
[3.4108407497406006, 0.0030012130737304688, 0.0, 3.223478078842163, 0.05709671974182129, 5.20122218132019, 1.3942623138427734, 0.015013456344604492]
[4.917867660522461, 0.0040018558502197266, 0.0, 4.832986354827881, 0.08512330055236816, 7.818122386932373, 2.069880723953247, 0.03403067588806152]
[6.808576583862305, 0.0050013065338134766, 0.0, 6.424399137496948, 0.11214613914489746, 10.428500413894653, 2.747067451477051, 0.0470428466796875]
[8.32053518295288, 0.006002187728881836, 0.0, 8.016862392425537, 0.14112520217895508, 13.03839659690857, 3.4226791858673096, 0.05805325508117676]
[10.238825559616089, 0.006002187728881836, 0.0, 9.61528491973877, 0.16180038452148438, 15.61980652809143, 4.115321397781372, 0.07706999778747559]
[11.76326322555542, 0.006002187728881836, 0.0, 11.211738586425781, 0.1878